```
사진에 있는 선반과 물체들을 감지하는 기능
ㄴ 선반을 사직으로 찍게 할건지 선반의 종류를 사용자가 선택하게 할건지
감지된 물체중에 사격형이 아닌 물체를 사각화 시켜주는 기능

사각화 된 물체를 종류에 맞게 분리해주는 기능

분리된 물체를 선반에 맞춰 정리해주는기능

정리된 모습의 물체화 선반을 단말기에 출력하는 기능
```

In [112]:
from ultralytics import YOLO
from ultralytics.engine.results import Results

# Load a pretrained YOLOv8n model
model = YOLO('yolov8x-seg.pt')

# Run inference on an image
results: Results = model('예제선반1.jpg')  # results list


image 1/1 l:\2024-1\SW\0429   \Arrangement_of_Jinn_Dolsoe\1.jpg: 480x640 4 bottles, 1 cup, 547.2ms
Speed: 2.0ms preprocess, 547.2ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)


In [113]:
class YoloObject:
    def __init__(self, object, xyxy, conf):
        self.object = object
        self.xyxy = xyxy
        self.conf = conf

    def show(self):
        print(f"xyxy: {self.xyxy}")
        print(f"conf: {self.conf}")

In [114]:
for r in results[0]:
    print(r.boxes.xyxy)

import cv2

# Global variables
drawing = False # true if mouse is pressed
ix, iy = -1, -1
img = None
temp_img = None

# Mouse callback function
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, img, temp_img

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
        temp_img = img.copy()

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing is True:
            img = temp_img.copy()
            cv2.rectangle(img, (ix, iy), (x, y), (0, 255, 0), 1)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img, (ix, iy), (x, y), (0, 255, 0), -1)

# Load an image
img = results[0].orig_img.copy()
cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_rectangle)

while(1):
    # # Define the new resolution
    # new_width = 800
    # new_height = 600

    # # Resize the image
    # resized_image = cv2.resize(img, (new_width, new_height))
    # cv2.imshow('image', resized_image)
    cv2.imshow('image', img)
    if cv2.waitKey(1) & 0xFF == 27:  # exit if ESC is pressed
        break

cv2.destroyAllWindows()

tensor([[422.8280, 531.7388, 534.7506, 853.8667]])
tensor([[604.0400, 408.5475, 729.7561, 839.8859]])
tensor([[331.1500, 664.9312, 441.1597, 837.7330]])
tensor([[509.1806, 346.8644, 647.1357, 806.8654]])
tensor([[247.2962, 643.5244, 301.4925, 842.6123]])


In [119]:
boxes = list()

# View results
for r in results[0]:
    boxes.append(YoloObject(r, r.boxes.xyxy, r.boxes.conf))
    # print(r.boxes)  # print the Boxes object containing the detection bounding boxes
    # print(r.boxes.xyxy)  # print the xyxy tensor of bounding boxes
    # print(r.boxes.conf)  # print the confidence tensor of bounding boxes

results[0].show()
# boxes[7].show()
boxes[2].object.show()
print(boxes[2].object.masks.xy)

arr = boxes[2].object.masks.xy

[array([[        342,         672],
       [        340,         674],
       [        338,         674],
       [        338,         678],
       [        336,         680],
       [        336,         826],
       [        340,         830],
       [        346,         830],
       [        348,         832],
       [        352,         832],
       [        354,         834],
       [        358,         834],
       [        360,         836],
       [        368,         836],
       [        370,         838],
       [        406,         838],
       [        408,         836],
       [        412,         836],
       [        414,         834],
       [        414,         832],
       [        422,         824],
       [        422,         822],
       [        424,         820],
       [        424,         816],
       [        426,         814],
       [        426,         812],
       [        428,         810],
       [        428,         808],
       [        430

imread로 원래 있던 그림을 기본으로 하고 float32형 array배열의 arr안에 있는 데이터를 점으로 cv2로 찍고싶어

In [121]:
import cv2

# Read the original image using imread
image = cv2.imread("예제선반1.jpg")

# Iterate over the data points in arr and draw them as points on the image
for point in arr[0]:
    print(point)
    x, y = point
    cv2.circle(image, (int(x), int(y)), radius=2, color=(0, 0, 255), thickness=-1)

# Display the image with the plotted points
cv2.imshow("Image with Points", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[        342         672]
[        340         674]
[        338         674]
[        338         678]
[        336         680]
[        336         826]
[        340         830]
[        346         830]
[        348         832]
[        352         832]
[        354         834]
[        358         834]
[        360         836]
[        368         836]
[        370         838]
[        406         838]
[        408         836]
[        412         836]
[        414         834]
[        414         832]
[        422         824]
[        422         822]
[        424         820]
[        424         816]
[        426         814]
[        426         812]
[        428         810]
[        428         808]
[        430         806]
[        430         794]
[        432         792]
[        432         776]
[        434         774]
[        434         766]
[        436         764]
[        436         750]
[        438         748]
[        438         702]
[        440

각각 object들의 좌표를 인식하고 물건이 배치되어있는 이미지 형태로 출력할 수 있게 해줘

In [117]:
class Shelf:
    def __init__(self, xyxy):
        self.xyxy = xyxy
        self.objects = []

    def add_object(self, obj):
        self.objects.append(obj)

    def remove_object(self, obj):
        self.objects.remove(obj)

    def display_objects(self):
        for obj in self.objects:
            print(obj)

    def clear_shelf(self):
        self.objects = []

In [126]:
image_path = '예제선반1.jpg'
image = cv2.imread(image_path)

# max_width = 500 # 최대 이미지 설정
# max_height = 500

for r in results:
    for idx, box in enumerate(r.boxes.xyxy):
        x1, y1, x2, y2 = map(int, box[:4])  # 경계 상자 좌표 (정수로 변환)
        cropped_img = image[y1:y2, x1:x2]  # 경계 상자 좌표를 사용하여 이미지 자르기

        height, width = cropped_img.shape[:2]

        # if width > max_width or height > max_height: # 이미지가 너무 크게 나와서 조절하는용도
        #     scaling_factor = min(max_width / width, max_height / height)
        # new_width = int(width * scaling_factor)
        new_width = int(width)
        # new_height = int(height * scaling_factor)
        new_height = int(height)
        cropped_img = cv2.resize(cropped_img, (new_width, new_height))
            
        print(f'Detected Object {idx+1} width: {new_width}, height: {new_height}')  # 객체의 가로와 세로 길이 출력
        cv2.imshow(f'Detected Object {idx+1}', cropped_img)
        cv2.waitKey(0)

cv2.destroyAllWindows() # 이미지창 닫기

Detected Object 1 width: 112, height: 322
Detected Object 2 width: 125, height: 431
Detected Object 3 width: 110, height: 173
Detected Object 4 width: 138, height: 460
Detected Object 5 width: 54, height: 199


물건의 가로와 세로 픽셀 길이로 선반에 들어가는지 가용성을 체크한다.